In [ ]:
!pip install yfinance

In [ ]:
# IMPORTS
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

QUESTION 1

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = " https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

filling_dfs = pd.read_html(response.text)

In [ ]:
filling_df = filling_dfs[0]
filling_df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
1,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
2,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
3,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
4,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
...,...,...,...,...,...
322,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
323,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
324,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
325,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [ ]:
filling_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     327 non-null    object
 1   Symbol          327 non-null    object
 2   Company Name    327 non-null    object
 3   Price Range     327 non-null    object
 4   Shares Offered  327 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [ ]:
# convert "Filling Date" to datetime
filling_df['Filing Date'] = pd.to_datetime(filling_df['Filing Date'])

In [ ]:
filling_df['Shares Offered'] = pd.to_numeric(filling_df['Shares Offered'], errors='coerce')

In [ ]:
# Function to calculate average price

def calculate_avg_price(price_range):
    if price_range == "-":
        return float('NaN')
    elif '-' in price_range:
        prices = price_range.split(' - ')
        return (float(prices[0][1:]) + float(prices[1][1:])) / 2
    else:
        return float(price_range[1:])

# Define new field 'Avg_price'
filling_df['Avg_price'] = filling_df['Price Range'].apply(calculate_avg_price)

In [ ]:
# Define new field 'Shares_offered_value'
filling_df['Shares_offered_value'] = filling_df["Avg_price"] * filling_df["Shares Offered"]


In [ ]:
filling_df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00,11250000.0
1,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00,60000000.0
2,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
3,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
4,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.00,NaN
...,...,...,...,...,...,...,...
322,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00,13500000.0
323,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00,50000000.0
324,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00,22500000.0
325,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00,17200000.0


In [ ]:
subset2023 = filling_df[(filling_df["Filing Date"] >='2023-01-01') & (filling_df["Filing Date"] <='2023-12-31')]
subset2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.0,4800000.0
51,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN,NaN,NaN
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667.0,7.5,8000002.5
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000.0,4.5,9900000.0
54,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...
164,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000.0,4.5,8437500.0
165,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000.0,5.0,16500000.0
166,2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,NaN,NaN,NaN
167,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000.0,4.5,16875000.0


In [ ]:
# Add Column for Day of the week name

subset2023["Day of the week"] = subset2023["Filing Date"].dt.day_name()


In [ ]:
subset2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value,Day of the week
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.0,4800000.0,Friday
51,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN,NaN,NaN,Friday
52,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667.0,7.5,8000002.5,Thursday
53,2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000.0,4.5,9900000.0,Wednesday
54,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN,NaN,NaN,Friday
...,...,...,...,...,...,...,...,...
164,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000.0,4.5,8437500.0,Tuesday
165,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000.0,5.0,16500000.0,Tuesday
166,2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,NaN,NaN,NaN,Monday
167,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000.0,4.5,16875000.0,Friday


In [ ]:
# Subset for Firday's Transaction only
subset2023_fri = subset2023[subset2023["Day of the week"] == "Friday"]
subset2023_fri.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value,Day of the week
50,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.0,4800000.0,Friday
51,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN,NaN,NaN,Friday
54,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN,NaN,NaN,Friday
55,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,NaN,NaN,NaN,Friday
60,2023-12-15,GIT,Going International Holding Company Limited,-,NaN,NaN,NaN,Friday


In [ ]:
# Get Sum of ShareD Offer Value for Friday in N'm
(subset2023_fri["Shares_offered_value"].sum()) / 1000000

285.7

QUESTION 2

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}


In [ ]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [ ]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [ ]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [ ]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='mixed')

In [ ]:
subset = stacked_ipos_df[(stacked_ipos_df["IPO Date"] < '2024-03-01')]
subset

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
33,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.36,149.00%
34,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.16,-96.16%
35,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.17,1.70%
36,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.70,-70.77%
37,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.24,-69.50%
...,...,...,...,...,...,...
213,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.16,-74.16%
214,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.47,-90.68%
215,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$35.77,138.47%
216,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.94,9.40%


In [ ]:
stacked_ipos_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-01,VIK,Viking Holdings Ltd.,$24.00,$26.99,16.21%
1,2024-04-26,ZONE,"CleanCore Solutions, Inc.",$4.00,$3.02,-24.63%
2,2024-04-25,RBRK,"Rubrik, Inc.",$32.00,$33.91,5.97%
3,2024-04-25,LOAR,Loar Holdings Inc.,$28.00,$46.40,71.43%
4,2024-04-25,MRX,Marex Group plc,$19.00,$18.85,-1.26%
...,...,...,...,...,...,...
213,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.16,-74.16%
214,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.47,-90.68%
215,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$35.77,138.47%
216,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.94,9.40%


In [ ]:
tickerList = list(subset["Symbol"].unique())
tickerList.remove('RYZB')
index_replace = tickerList.index("PTHR")
tickerList[index_replace] = "PTHRF"

In [ ]:
def get_growth_df(df:pd.DataFrame, prefix:str)->pd.DataFrame:
  for i in range(1, 31, 1):
    df['growth_'+prefix+'_'+str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
    GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

def get_days_max(df):
  largest_growth = 0
  growth_days = ""
  for growth in growth_df.columns:
    percentile_75 = growth_df[growth].quantile(0.75)
    if percentile_75 >= largest_growth:
      largest_growth = percentile_75
      growth_days = growth
  xd = growth_days.split("_")
  days = xd[2][0:-1]
  return (days, largest_growth)

In [ ]:
res_growth = 0
res_days = ""
res_ticker = ""
for ticker in tickerList:
  df_ticker = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")
  growth_df = get_growth_df(df_ticker, ticker)
  max_days, large_growth = get_days_max(growth_df)
  if large_growth >= res_growth:
      res_growth = large_growth
      res_days = max_days
      res_ticker = ticker
      print(res_growth, res_days, res_ticker)
print("Final",res_growth, res_days, res_ticker)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1.973075262535216 27 SMXT



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

Final 1.973075262535216 27 SMXT


QUESTION 3

In [ ]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [ ]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [ ]:
def get_growth_df(df:pd.DataFrame, prefix:str)->pd.DataFrame:
  i = 7
  df['growth_'+prefix+'_'+str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
  GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

def preproc(stockList):
  df_ticker = yf.download(tickers = stockList[0],
                      period = "max",
                      interval = "1d")

  df_ticker = df_ticker[(df_ticker.index >= '2014-01-01') & (df_ticker.index <= '2023-12-31')]

  final_growth_7d =  get_growth_df(df_ticker, stockList[0])

  for ticker in stockList[1:]:
    df_ticker = yf.download(tickers = ticker,
                      period = "max",
                      interval = "1d")
    df_ticker = df_ticker[(df_ticker.index >= '2014-01-01') & (df_ticker.index <= '2023-12-31')]

    tempt_growth_7d =  get_growth_df(df_ticker, ticker)
    final_growth_7d = pd.concat([final_growth_7d, tempt_growth_7d], axis = 1)
  return final_growth_7d

In [ ]:
result_LARGEST_STOCKS = preproc(LARGEST_STOCKS)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
result_LARGEST_STOCKS.tail()


,growth_MSFT_7d,growth_AAPL_7d,growth_GOOG_7d,growth_NVDA_7d,growth_AMZN_7d,growth_META_7d,growth_BRK-B_7d,growth_LLY_7d,growth_AVGO_7d,growth_V_7d,...,growth_TCS.NS_7d,growth_HDB_7d,growth_BHARTIARTL.NS_7d,growth_IBN_7d,growth_SBIN.NS_7d,growth_LICI.NS_7d,growth_INFY_7d,growth_ITC.NS_7d,growth_HINDUNILVR.NS_7d,growth_LT.NS_7d
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,1.000561,0.977975,1.065313,1.015430,1.030771,1.055715,0.984642,0.954628,1.034367,0.984945,...,1.064129,1.016384,0.989649,0.964763,1.027265,0.984784,1.073841,0.999012,1.025359,1.022987
2023-12-26,1.023857,0.974459,1.072222,1.019214,1.040632,1.065012,0.996398,0.994771,1.027659,1.001662,...,1.034985,0.997608,0.994678,0.967585,1.023090,0.958356,1.004355,0.992067,1.027182,1.016588
2023-12-27,1.009009,0.977628,1.056784,1.010779,1.022471,1.068404,1.001262,1.016555,1.001424,1.003488,...,0.987102,1.014528,1.028865,0.978411,1.000463,1.031715,0.996251,0.997599,1.033850,1.016055
2023-12-28,1.007058,0.988208,1.029813,0.988917,0.995522,1.039754,0.994217,1.001880,0.983061,1.007857,...,0.984634,1.016483,1.046907,0.986960,1.004472,1.031498,0.987234,1.027566,1.038769,1.007575
2023-12-29,1.007448,0.977607,1.020492,0.998347,0.987971,1.010275,0.985793,1.005364,0.979528,1.001385,...,0.994025,1.013899,1.044261,0.973459,0.979631,1.048423,0.978180,1.013155,1.040321,1.007731


In [ ]:
result_LARGE_STOCKS = preproc(LARGE_STOCKS)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
result_LARGE_STOCKS.tail()

,growth_PRX.AS_7d,growth_CDI.PA_7d,growth_AIR.PA_7d,growth_SU.PA_7d,growth_ETN_7d,growth_SNY_7d,growth_BUD_7d,growth_DTE.DE_7d,growth_ALV.DE_7d,growth_MDT_7d,...,growth_MARUTI.NS_7d,growth_HCLTECH.NS_7d,growth_TATAMOTORS.NS_7d,growth_SUNPHARMA.NS_7d,growth_ONGC.NS_7d,growth_ADANIENT.NS_7d,growth_ADANIENT.NS_7d,growth_NTPC.NS_7d,growth_KOTAKBANK.NS_7d,growth_TITAN.NS_7d
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,0.861223,0.997884,0.979902,1.024921,1.007813,1.009110,1.019118,0.965640,0.990406,1.008623,...,0.984306,1.068288,1.006109,1.008556,1.055915,0.976800,0.976800,1.029757,1.018782,1.006940
2023-12-26,NaN,NaN,NaN,NaN,1.010560,1.017413,1.029145,NaN,NaN,0.994719,...,0.992027,1.030786,0.999722,1.013157,1.058178,0.990118,0.990118,1.048070,1.017801,1.018182
2023-12-27,0.871803,0.973684,0.995430,1.016201,1.014363,1.043414,1.037754,0.995860,1.011297,1.006058,...,1.000204,0.987092,1.011606,1.013514,1.022382,0.950381,0.950381,1.003114,1.027964,1.024635
2023-12-28,0.879645,0.982578,1.001147,1.001215,1.015266,1.017305,1.029276,1.001158,1.001873,1.004370,...,0.995349,0.978827,1.031609,1.007504,1.046734,0.942730,0.942730,1.035290,1.040283,1.026384
2023-12-29,0.924302,0.990203,1.004311,1.004198,1.011211,1.013657,1.011901,1.009984,1.005193,1.003777,...,1.006718,0.985084,1.069964,1.011647,1.023714,0.968619,0.968619,1.004844,1.033332,1.018003


In [ ]:
# Average column wise
average_LARGEST_STOCKS = result_LARGEST_STOCKS.mean(axis = 1)
average_LARGEST_STOCKS
#np.mean(average)

Date
2014-01-01         NaN
2014-01-02         NaN
2014-01-03         NaN
2014-01-06         NaN
2014-01-07         NaN
                ...   
2023-12-22    1.013788
2023-12-26    1.014900
2023-12-27    1.014155
2023-12-28    1.008533
2023-12-29    1.005337
Length: 2595, dtype: float64

In [ ]:
# Average column wise
average_LARGE_STOCKS = result_LARGE_STOCKS.mean(axis = 1)
average_LARGE_STOCKS
#np.mean(average)

Date
2014-01-01         NaN
2014-01-02         NaN
2014-01-03         NaN
2014-01-06         NaN
2014-01-07         NaN
                ...   
2023-12-22    1.003014
2023-12-26    1.013505
2023-12-27    1.002167
2023-12-28    0.998583
2023-12-29    0.999313
Length: 2595, dtype: float64

In [ ]:
final_df = pd.concat([average_LARGE_STOCKS, average_LARGEST_STOCKS], axis = 1)
final_df.columns = ["LARGE_STOCKS", "LARGEST_STOCKS"]

In [ ]:
final_df.tail()

,LARGE_STOCKS,LARGEST_STOCKS
Date,,
2023-12-22,1.003014,1.013788
2023-12-26,1.013505,1.014900
2023-12-27,1.002167,1.014155
2023-12-28,0.998583,1.008533
2023-12-29,0.999313,1.005337


In [ ]:
# Define a function to perform the subtraction and map the result to 1 or 0
def subtract_and_map(final_df):
    result = final_df['LARGE_STOCKS'] - final_df['LARGEST_STOCKS']
    if result > 0:
        return 1
    else:
        return 0

# Apply the function to create the new column
final_df['Result'] = final_df.apply(subtract_and_map, axis=1)

round(final_df['Result'].value_counts()[1] / 2595 * 100)

47

QUESTION 4

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/stocks_df_combined_trunc_2014_2023.parquet.brotli')

#df.reset_index(drop=True, inplace=True)
#cci is difference between the current price and the historical average price.

In [ ]:
df_ss = df[(df["Month"] >= "2014-01-01") & (df["Month"] <= "2023-12-31")]


In [ ]:
df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
1,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
2,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
3,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
4,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(df)

80762

In [ ]:
def get_growth_df(df:pd.DataFrame, prefix:str)->pd.DataFrame:
  i = 5
  df['growth_'+prefix+'_'+str(i)+'d'] = df['Adj Close_x'] / df['Adj Close_x'].shift(i)
  GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

def profit_calc(sub_df, ticker, i):
  get_growth_df(sub_df, ticker)
  grossProfit = 0
  investableAmt = 1000

  # Loop through the DataFrame
  for index, row in sub_df.iterrows():
    # Check if the current day of the week is Friday
    if row['Weekday'] == 4:
      # Get the subset of the DataFrame from the beginning to the current row
      subset = sub_df.iloc[:index+1]
      #print("index", index)
      # Calculate the CCI until the current Friday
      avg_cci_until_friday = subset['Adj Close_x'].mean()

      if (sub_df["Adj Close_x"].loc[index] - avg_cci_until_friday) > 200:
        shareBoughtNo = investableAmt / (sub_df["Adj Close_x"].loc[index])

        #newAmount = (sub_df["Adj Close_x"].loc[index]) * sub_df['growth_'+ticker+'_'+str(i)+'d'].loc[index] * shareBoughtNo
        if (index+5) < len(sub_df):
          #newAmount = (sub_df["Adj Close_x"].loc[index]) * sub_df['growth_'+ticker+'_'+str(i)+'d'].loc[index] * shareBoughtNo
          #newAmount = ((sub_df["Adj Close_x"].loc[index])) - (sub_df["Adj Close_x"].loc[index+5])* shareBoughtNo
          newAmount = (sub_df["Adj Close_x"].loc[index+5])* shareBoughtNo
          #print(shareBoughtNo, newAmount)
          profit = newAmount - investableAmt
          #print(profit)
          grossProfit = grossProfit + profit
          #print(grossProfit, ticker)

  return grossProfit

In [ ]:
finalProfit = []
for ticker in list(df.Ticker.unique()):
  sub_df = df[df["Ticker"] == ticker]

  df_sorted = sub_df.sort_values(by='Date')
  df_sorted.reset_index(drop=True, inplace=True)
  #df_sorted2 = df_sorted.set_index('Date')

  result = profit_calc(df_sorted, ticker, 5)
  finalProfit.append([ticker, result])
grossTable = pd.DataFrame(finalProfit, columns = ["Ticker", "Ticker_Profit"])
grossTable

,Ticker,Ticker_Profit
0,MSFT,-58.086120
1,AAPL,0.000000
2,GOOG,0.000000
3,NVDA,516.415200
4,AMZN,0.000000
5,META,-106.606850
6,BRK-B,0.000000
7,LLY,368.795072
8,AVGO,979.417594
9,V,0.000000


In [ ]:
totalInvestmentProfit = grossTable["Ticker_Profit"].sum()
round(totalInvestmentProfit/1000)

10